In [2]:
import pandas as pd
import pyodbc
# online_retail = pd.read_excel('Online Retail.xlsx')
# online_retail.rename(columns=str.lower).dropna(subset=['customerid']).astype({'customerid': int}).to_csv('online_retail_processed.csv', index=False)
df = pd.read_csv('online_retail_processed.csv')
df = df.astype({'invoicedate': 'datetime64[ns]'})
df['first_invoice_date'] = df.groupby('customerid')['invoicedate'].transform('min')
df['cohort'] = df.first_invoice_date.dt.to_period('M')
df = df[~df.cohort.isin(['2010-12', '2011-12'])]

df['month_diff'] = (df.invoicedate.dt.to_period('M') - df.cohort).apply(lambda x: x.n)

In [6]:
transactions_trunc = df[['customerid', 'cohort', 'month_diff']].drop_duplicates()

In [ ]:
transactions_trunc.head()


,customerid,cohort,month_diff
0,17850,2010-12,0
9,13047,2010-12,0
26,12583,2010-12,0
46,13748,2010-12,0
65,15100,2010-12,0


In [10]:
transactions_trunc.pivot_table(columns = 'month_diff', 
                                index = 'cohort', values = 'customerid',  aggfunc = lambda x: x.nunique())

month_diff,0,1,2,3,4,5,6,7,8,9,10,11,12
cohort,,,,,,,,,,,,,
2010-12,948.0,362.0,317.0,367.0,341.0,376.0,360.0,336.0,336.0,374.0,354.0,474.0,260.0
2011-01,421.0,101.0,119.0,102.0,138.0,126.0,110.0,108.0,131.0,146.0,155.0,63.0,NaN
2011-02,380.0,94.0,73.0,106.0,102.0,94.0,97.0,107.0,98.0,119.0,35.0,NaN,NaN
2011-03,440.0,84.0,112.0,96.0,102.0,78.0,116.0,105.0,127.0,39.0,NaN,NaN,NaN
2011-04,299.0,68.0,66.0,63.0,62.0,71.0,69.0,78.0,25.0,NaN,NaN,NaN,NaN
2011-05,279.0,66.0,48.0,48.0,60.0,68.0,74.0,29.0,NaN,NaN,NaN,NaN,NaN
2011-06,235.0,49.0,44.0,64.0,58.0,79.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN
2011-07,191.0,40.0,39.0,44.0,52.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08,167.0,42.0,42.0,42.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
df.cohort.dt.toordinal()

AttributeError: 'PeriodProperties' object has no attribute 'toordinal'

In [77]:
df.cohort[0].ordinal

491